In [1]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

In [2]:
import pandas as pd

## Importamos la información

In [3]:
path = 's3://inai-summerofdata/mlpreproc/inai.parquet'
df = pd.read_parquet(path, engine='pyarrow')
pd.options.display.max_columns = None

In [4]:
df.head()

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector,tipo_archivo_respuesta,cantidad_archivos_respuesta,texto_respuesta_adjunto,respuestareal
0,1857500001912,2012-01-01 05:04:07,pemex exploracion y produccion,terminada,electronica,informacion publica,copia de los documentos y los con que cuenta a...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 02...,entrega de informacion en medio electronico
1,1857500002012,2012-01-01 05:13:16,pemex exploracion y produccion,terminada,electronica,informacion publica,del informacion y usados en la de flujo de efe...,None,None,entrega por internet en el infomex,2012-03-02,entrega de informacion en medio electronico,se anexa archivo con la respuesta a la informa...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-03-02,mexico,distrito federal,benito juarez,3100,energia,pdf,1.0,pemex exploracion y produccion oficio fecha 01...,entrega de informacion en medio electronico
2,0001800000712,2012-01-01 09:46:39,secretaria de energia,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,inexistencia de la informacion solicitada,el asunto se turno a la direccion general de r...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,energia,zip,2.0,None,inexistencia de la informacion solicitada
3,0000600003712,2012-01-01 10:11:41,secretaria de hacienda y credito publico,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,la informacion esta disponible publicamente,direcci&oacute;n de internet en donde se encue...,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-02,mexico,distrito federal,miguel hidalgo,11230,hacienda y credito publico,pdf,1.0,inidos secretaria de hacienda y credito public...,la informacion esta disponible publicamente
4,0001500002812,2012-01-01 10:32:15,secretaria de desarrollo agrario territorial y...,terminada,electronica,informacion publica,se me proporcione en magnetico y por este medi...,None,None,entrega por internet en el infomex,2012-02-02,entrega de informacion en medio electronico,None,https://www.infomex.org.mx/gobiernofederal/mod...,2012-02-01,mexico,distrito federal,miguel hidalgo,11230,reforma agraria,zip,4.0,None,sin respuesta


## Implementación LDA

Extracción de tópicos para textos libres de solicitudes.

Nos quedaremos con la columna descripcionsolictud de nuestro dataframe.

In [5]:
data = df[['descripcionsolicitud']]
data['index'] = data.index
docs = data

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
len(docs)

1559012

In [7]:
docs.head()

,descripcionsolicitud,index
0,copia de los documentos y los con que cuenta a...,0
1,del informacion y usados en la de flujo de efe...,1
2,se me proporcione en magnetico y por este medi...,2
3,se me proporcione en magnetico y por este medi...,3
4,se me proporcione en magnetico y por este medi...,4


In [8]:
texto = docs[docs['index'] == 4310].values[0][0]
texto

'a esta institucion solicito sea explicado el procedimiento el cual se instalaron cafeterias en los planteles al igual que maquinas de cafe refrescos y otros productos industrializados. tambien solicito copia de los contratos convenios a de los se formaliza instalacion en los citados centros . ganancias que a la institucion (mes por en uno de los planteles) deja el haber consentido la instalacion-operacion-funcionamiento de tales formas mercantiles. con fueron negociadas y que autoridades dieron consentimiento? y cual el destino de esos recursos?'

### Pre-Procesamiento de la data

#### Tokenización

In [8]:
import spacy
nlp = spacy.load('es_core_news_sm')

In [9]:
texto = docs[docs['index'] == 4310].values[0][0]
texto

'a esta institucion solicito sea explicado el procedimiento el cual se instalaron cafeterias en los planteles al igual que maquinas de cafe refrescos y otros productos industrializados. tambien solicito copia de los contratos convenios a de los se formaliza instalacion en los citados centros . ganancias que a la institucion (mes por en uno de los planteles) deja el haber consentido la instalacion-operacion-funcionamiento de tales formas mercantiles. con fueron negociadas y que autoridades dieron consentimiento? y cual el destino de esos recursos?'

In [11]:
doc = nlp(texto)
doc

a esta institucion solicito sea explicado el procedimiento el cual se instalaron cafeterias en los planteles al igual que maquinas de cafe refrescos y otros productos industrializados. tambien solicito copia de los contratos convenios a de los se formaliza instalacion en los citados centros . ganancias que a la institucion (mes por en uno de los planteles) deja el haber consentido la instalacion-operacion-funcionamiento de tales formas mercantiles. con fueron negociadas y que autoridades dieron consentimiento? y cual el destino de esos recursos?

In [12]:
tokens = [t.orth_ for t in doc]
tokens

['a',
 'esta',
 'institucion',
 'solicito',
 'sea',
 'explicado',
 'el',
 'procedimiento',
 'el',
 'cual',
 'se',
 'instalaron',
 'cafeterias',
 'en',
 'los',
 'planteles',
 'al',
 'igual',
 'que',
 'maquinas',
 'de',
 'cafe',
 'refrescos',
 'y',
 'otros',
 'productos',
 'industrializados',
 '.',
 'tambien',
 'solicito',
 'copia',
 'de',
 'los',
 'contratos',
 'convenios',
 'a',
 'de',
 'los',
 'se',
 'formaliza',
 'instalacion',
 'en',
 'los',
 'citados',
 'centros',
 '.',
 'ganancias',
 'que',
 'a',
 'la',
 'institucion',
 '(',
 'mes',
 'por',
 'en',
 'uno',
 'de',
 'los',
 'planteles',
 ')',
 'deja',
 'el',
 'haber',
 'consentido',
 'la',
 'instalacion-operacion-funcionamiento',
 'de',
 'tales',
 'formas',
 'mercantiles',
 '.',
 'con',
 'fueron',
 'negociadas',
 'y',
 'que',
 'autoridades',
 'dieron',
 'consentimiento',
 '?',
 'y',
 'cual',
 'el',
 'destino',
 'de',
 'esos',
 'recursos',
 '?']

#### Limpieza del texto - tokenización

In [13]:
import spacy
nlp = spacy.load('es_core_news_sm')

In [14]:
texto = docs[docs['index'] == 4310].values[0][0]
texto

'a esta institucion solicito sea explicado el procedimiento el cual se instalaron cafeterias en los planteles al igual que maquinas de cafe refrescos y otros productos industrializados. tambien solicito copia de los contratos convenios a de los se formaliza instalacion en los citados centros . ganancias que a la institucion (mes por en uno de los planteles) deja el haber consentido la instalacion-operacion-funcionamiento de tales formas mercantiles. con fueron negociadas y que autoridades dieron consentimiento? y cual el destino de esos recursos?'

In [15]:
doc = nlp(texto)
doc

a esta institucion solicito sea explicado el procedimiento el cual se instalaron cafeterias en los planteles al igual que maquinas de cafe refrescos y otros productos industrializados. tambien solicito copia de los contratos convenios a de los se formaliza instalacion en los citados centros . ganancias que a la institucion (mes por en uno de los planteles) deja el haber consentido la instalacion-operacion-funcionamiento de tales formas mercantiles. con fueron negociadas y que autoridades dieron consentimiento? y cual el destino de esos recursos?

Quitamos puntos y stopwords.

In [16]:
lexical_tokens = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
lexical_tokens

['a',
 'institucion',
 'solicito',
 'explicado',
 'procedimiento',
 'instalaron',
 'cafeterias',
 'planteles',
 'maquinas',
 'cafe',
 'refrescos',
 'y',
 'productos',
 'industrializados',
 'solicito',
 'copia',
 'contratos',
 'convenios',
 'a',
 'formaliza',
 'instalacion',
 'citados',
 'centros',
 'ganancias',
 'a',
 'institucion',
 'mes',
 'planteles',
 'deja',
 'consentido',
 'instalacion-operacion-funcionamiento',
 'tales',
 'formas',
 'mercantiles',
 'negociadas',
 'y',
 'autoridades',
 'consentimiento',
 'y',
 'destino',
 'recursos']

#### Normalizar texto

In [17]:
words = [t.lower() for t in lexical_tokens if len(t) > 3 and t.isalpha()]
words

['institucion',
 'solicito',
 'explicado',
 'procedimiento',
 'instalaron',
 'cafeterias',
 'planteles',
 'maquinas',
 'cafe',
 'refrescos',
 'productos',
 'industrializados',
 'solicito',
 'copia',
 'contratos',
 'convenios',
 'formaliza',
 'instalacion',
 'citados',
 'centros',
 'ganancias',
 'institucion',
 'planteles',
 'deja',
 'consentido',
 'tales',
 'formas',
 'mercantiles',
 'negociadas',
 'autoridades',
 'consentimiento',
 'destino',
 'recursos']

### Creamos una función con que haga todo lo anterior - Tokenización (Normalizar texto)

In [18]:
def NormalizarTexto(texto):
    doc = nlp(texto)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and t.isalpha()]
    
    return lexical_tokens

In [19]:
word_list = NormalizarTexto(texto)
word_list

['institucion',
 'solicito',
 'explicado',
 'procedimiento',
 'instalaron',
 'cafeterias',
 'planteles',
 'maquinas',
 'cafe',
 'refrescos',
 'productos',
 'industrializados',
 'solicito',
 'copia',
 'contratos',
 'convenios',
 'formaliza',
 'instalacion',
 'citados',
 'centros',
 'ganancias',
 'institucion',
 'planteles',
 'deja',
 'consentido',
 'tales',
 'formas',
 'mercantiles',
 'negociadas',
 'autoridades',
 'consentimiento',
 'destino',
 'recursos']

#### Lematización

In [20]:
import spacy
nlp = spacy.load('es_core_news_sm')

In [21]:
doc = nlp(texto)
doc

a esta institucion solicito sea explicado el procedimiento el cual se instalaron cafeterias en los planteles al igual que maquinas de cafe refrescos y otros productos industrializados. tambien solicito copia de los contratos convenios a de los se formaliza instalacion en los citados centros . ganancias que a la institucion (mes por en uno de los planteles) deja el haber consentido la instalacion-operacion-funcionamiento de tales formas mercantiles. con fueron negociadas y que autoridades dieron consentimiento? y cual el destino de esos recursos?

In [22]:
lemmas = [tok.lemma_.lower() for tok in doc]
lemmas

['a',
 'este',
 'institucion',
 'solicitar',
 'ser',
 'explicar',
 'el',
 'procedimiento',
 'el',
 'cual',
 'se',
 'instalar',
 'cafeterias',
 'en',
 'lo',
 'plantel',
 'al',
 'igual',
 'que',
 'maquinar',
 'de',
 'cafe',
 'refresco',
 'y',
 'otro',
 'producto',
 'industrializar',
 '.',
 'tambien',
 'solicitar',
 'copiar',
 'de',
 'lo',
 'contrato',
 'convenio',
 'a',
 'de',
 'lo',
 'se',
 'formalizar',
 'instalacion',
 'en',
 'lo',
 'citar',
 'centro',
 '.',
 'ganancia',
 'que',
 'a',
 'lo',
 'institucion',
 '(',
 'mes',
 'por',
 'en',
 'unir',
 'de',
 'lo',
 'plantel',
 ')',
 'dejar',
 'el',
 'haber',
 'consentir',
 'lo',
 'instalacion-operacion-funcionamiento',
 'de',
 'talar',
 'formar',
 'mercantil',
 '.',
 'con',
 'ser',
 'negociar',
 'y',
 'que',
 'autoridad',
 'dar',
 'consentimiento',
 '?',
 'y',
 'cual',
 'el',
 'destinar',
 'de',
 'ese',
 'recurso',
 '?']

In [23]:
lemmas_no_pron = [tok.lemma_.lower() for tok in doc if tok.pos_ != 'PRON']
lemmas_no_pron

['a',
 'este',
 'institucion',
 'solicitar',
 'ser',
 'explicar',
 'el',
 'procedimiento',
 'el',
 'instalar',
 'cafeterias',
 'en',
 'lo',
 'plantel',
 'al',
 'igual',
 'que',
 'maquinar',
 'de',
 'cafe',
 'refresco',
 'y',
 'otro',
 'producto',
 'industrializar',
 '.',
 'tambien',
 'solicitar',
 'copiar',
 'de',
 'lo',
 'contrato',
 'convenio',
 'a',
 'de',
 'lo',
 'formalizar',
 'instalacion',
 'en',
 'lo',
 'citar',
 'centro',
 '.',
 'ganancia',
 'que',
 'a',
 'lo',
 'institucion',
 '(',
 'mes',
 'por',
 'en',
 'de',
 'lo',
 'plantel',
 ')',
 'dejar',
 'el',
 'haber',
 'consentir',
 'lo',
 'instalacion-operacion-funcionamiento',
 'de',
 'talar',
 'formar',
 'mercantil',
 '.',
 'con',
 'ser',
 'negociar',
 'y',
 'que',
 'autoridad',
 'dar',
 'consentimiento',
 '?',
 'y',
 'el',
 'destinar',
 'de',
 'ese',
 'recurso',
 '?']

### Radicalizacion (stemming) -- Obtener raíces

In [24]:
import nltk
from nltk import SnowballStemmer

In [25]:
spanishstemmer = SnowballStemmer('spanish')

**tokenizamos**

In [26]:
tokens = NormalizarTexto(texto)
tokens

['institucion',
 'solicito',
 'explicado',
 'procedimiento',
 'instalaron',
 'cafeterias',
 'planteles',
 'maquinas',
 'cafe',
 'refrescos',
 'productos',
 'industrializados',
 'solicito',
 'copia',
 'contratos',
 'convenios',
 'formaliza',
 'instalacion',
 'citados',
 'centros',
 'ganancias',
 'institucion',
 'planteles',
 'deja',
 'consentido',
 'tales',
 'formas',
 'mercantiles',
 'negociadas',
 'autoridades',
 'consentimiento',
 'destino',
 'recursos']

**steemming**

In [27]:
stems = [spanishstemmer.stem(token) for token in tokens]
stems

['institucion',
 'solicit',
 'explic',
 'proced',
 'instal',
 'cafeteri',
 'plantel',
 'maquin',
 'caf',
 'refresc',
 'product',
 'industrializ',
 'solicit',
 'copi',
 'contrat',
 'conveni',
 'formaliz',
 'instalacion',
 'cit',
 'centr',
 'gananci',
 'institucion',
 'plantel',
 'dej',
 'consent',
 'tal',
 'form',
 'mercantil',
 'negoci',
 'autor',
 'consent',
 'destin',
 'recurs']

# Creamos una función para tokenizar, normalizar y radicalizar

In [ ]:
import spacy
import nltk
from nltk import SnowballStemmer

In [ ]:
nlp = spacy.load('es_core_news_sm')
spanishstemmer = SnowballStemmer('spanish')

In [92]:
def RadicalizarTexto(texto):
    
    doc = nlp(texto)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and t.isalpha()]
    
    stems = [spanishstemmer.stem(token) for token in lexical_tokens]
    
    return stems

**Ejemplo**

In [31]:
doc_sample = docs[docs['index'] == 4310].values[0][0]
doc_sample

'a esta institucion solicito sea explicado el procedimiento el cual se instalaron cafeterias en los planteles al igual que maquinas de cafe refrescos y otros productos industrializados. tambien solicito copia de los contratos convenios a de los se formaliza instalacion en los citados centros . ganancias que a la institucion (mes por en uno de los planteles) deja el haber consentido la instalacion-operacion-funcionamiento de tales formas mercantiles. con fueron negociadas y que autoridades dieron consentimiento? y cual el destino de esos recursos?'

In [94]:
word_list = RadicalizarTexto(doc_sample)
word_list

['institucion',
 'solicit',
 'explic',
 'proced',
 'instal',
 'cafeteri',
 'plantel',
 'maquin',
 'caf',
 'refresc',
 'product',
 'industrializ',
 'solicit',
 'copi',
 'contrat',
 'conveni',
 'formaliz',
 'instalacion',
 'cit',
 'centr',
 'gananci',
 'institucion',
 'plantel',
 'dej',
 'consent',
 'tal',
 'form',
 'mercantil',
 'negoci',
 'autor',
 'consent',
 'destin',
 'recurs']

**Ahora lo haremos para todo el dataframe columna descripcionsolictud**

Paralelizaremos la tarea dado que es una tarea tardada.

In [33]:
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool

In [34]:
cores = multiprocessing.cpu_count()
pool = ThreadPool(cores)
cores

12

In [35]:
processed_docs = pool.map(RadicalizarTexto, docs['descripcionsolicitud'])
pool.close()

In [36]:
processed_docs[:10]

[['copi',
  'document',
  'comport',
  'histor',
  'tipic',
  'comprend',
  'proyect',
  'estudi',
  'cientif',
  'variabl',
  'present',
  'pemex',
  'exploracion',
  'produccion',
  'elabor',
  'evaluacion',
  'integral',
  'proyect',
  'shcp',
  'inclu',
  'form',
  'especif',
  'produccion',
  'aceit',
  'barril',
  'petrole',
  'poz',
  'consider',
  'evaluacion',
  'integral',
  'proyect',
  'declinacion',
  'anual',
  'petrole',
  'poz',
  'usad',
  'produccion',
  'preci',
  'petrole',
  'consider',
  'respons',
  'evaluacion',
  'integral',
  'proyect',
  'cost',
  'operacion',
  'anual',
  'petrole',
  'poz',
  'consider',
  'evaluacion',
  'integral',
  'proyect'],
 ['informacion',
  'usad',
  'fluj',
  'efect',
  'poz',
  'tipic',
  'agu',
  'fri',
  'elabor',
  'respond',
  'solicitud',
  'solicit',
  'produccion',
  'aceit',
  'consider',
  'tabl',
  'cost',
  'capital',
  'usad',
  'recuper',
  'mes',
  'preci',
  'aceit',
  'consider',
  'formul',
  'usad',
  'explicaci

In [37]:
len(processed_docs)

1559012

Creamos una bag of words de nuestro dataframe

In [38]:
import gensim

In [39]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [40]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 aceit
1 anual
2 barril
3 cientif
4 comport
5 comprend
6 consider
7 copi
8 cost
9 declinacion
10 document


Filtramos tokens que aparecen en:
+ Menos de 15 documentos o
+ mas que el 0.5 del total de documentos
+ despues de los dos pasos anteriores, manten solo los primeros 100,000 tokens mas frecuentes.

In [41]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

**Gensim doc2bow**

Para cada documento creamos un diccionario reportando cuantas palabras y cuantas veces esas palabras aparecen.

In [42]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [43]:
bow_corpus[4310]

[(7, 1),
 (16, 1),
 (49, 2),
 (57, 1),
 (113, 1),
 (151, 1),
 (173, 1),
 (330, 1),
 (344, 2),
 (415, 1),
 (629, 1),
 (835, 1),
 (931, 1),
 (1037, 1),
 (1195, 1),
 (1235, 1),
 (1495, 1),
 (1520, 1),
 (1775, 1),
 (1878, 1),
 (2194, 2),
 (2291, 1),
 (3605, 1),
 (4600, 2),
 (5042, 1),
 (5043, 1),
 (5044, 1),
 (5045, 1),
 (5046, 1)]

### TF-IDF

Creamos un modelo tf-idf.

In [44]:
from gensim import corpora, models
from pprint import pprint

In [45]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [46]:
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.13727626585513125),
 (1, 0.15434779418679048),
 (2, 0.13566125046935537),
 (3, 0.11316682203127554),
 (4, 0.1353721929203078),
 (5, 0.07640240866699256),
 (6, 0.23085975679552553),
 (7, 0.042564992460744445),
 (8, 0.06990324891288505),
 (9, 0.20347610783454645),
 (10, 0.04678664785497465),
 (11, 0.09030966762093849),
 (12, 0.06996941753765298),
 (13, 0.07118613529159919),
 (14, 0.3090214063527276),
 (15, 0.10248823450028925),
 (16, 0.06333358681952889),
 (17, 0.10045414506239277),
 (18, 0.05613741748448796),
 (19, 0.34432142196950205),
 (20, 0.08083015337642883),
 (21, 0.08103341375827121),
 (22, 0.37506103734380675),
 (23, 0.35700101417652275),
 (24, 0.0782102212060324),
 (25, 0.04625554318552719),
 (26, 0.26617333004311855),
 (27, 0.3148946924521641),
 (28, 0.06256905870717054),
 (29, 0.10842562043179677),
 (30, 0.18352006957047706),
 (31, 0.12067348741327555),
 (32, 0.12267008953849452)]


## Implementación LDA usando Bag of Words

In [47]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10,
                                      id2word=dictionary, passes=2,
                                      workers=cores)

Para cada topico exploraremos las palabras que contiene y sus pesos relativos.

In [48]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.030*"solicit" + 0.019*"informacion" + 0.017*"part" + 0.014*"presupuest" + 0.012*"federal" + 0.012*"entid" + 0.011*"inform" + 0.011*"desgl" + 0.011*"dependent" + 0.011*"fech"
Topic: 1 
Words: 0.027*"solicit" + 0.017*"mont" + 0.017*"fech" + 0.015*"pag" + 0.014*"servici" + 0.013*"informacion" + 0.013*"concept" + 0.013*"trabaj" + 0.011*"bas" + 0.011*"tip"
Topic: 2 
Words: 0.241*"solicitud" + 0.022*"salud" + 0.016*"informacion" + 0.014*"edad" + 0.012*"program" + 0.012*"person" + 0.011*"gener" + 0.011*"respuest" + 0.009*"solicit" + 0.009*"nin"
Topic: 3 
Words: 0.080*"medic" + 0.049*"medicament" + 0.039*"clav" + 0.028*"solicit" + 0.023*"unid" + 0.023*"atencion" + 0.022*"hospital" + 0.022*"pacient" + 0.022*"recet" + 0.018*"fech"
Topic: 4 
Words: 0.036*"informacion" + 0.028*"solicit" + 0.014*"solucion" + 0.011*"acces" + 0.011*"document" + 0.010*"articul" + 0.009*"cas" + 0.009*"transparent" + 0.009*"dich" + 0.008*"solicitud"
Topic: 5 
Words: 0.046*"informacion" + 0.038*"articu

## Implementación LDA usando TF-IDF

In [49]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                            num_topics=10,
                                            id2word=dictionary,
                                            passes=2,
                                            workers=cores)

In [50]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.012*"copi" + 0.011*"ofici" + 0.010*"vazquez" + 0.009*"nav" + 0.009*"contrat" + 0.006*"solicit" + 0.006*"servici" + 0.006*"consultor" + 0.005*"fech" + 0.005*"plag"
Topic: 1 Word: 0.961*"solicitud" + 0.006*"informacion" + 0.001*"suplenci" + 0.001*"escane" + 0.001*"googl" + 0.001*"impos" + 0.001*"consupag" + 0.001*"crohn" + 0.000*"land" + 0.000*"postur"
Topic: 2 Word: 0.009*"concesion" + 0.008*"municipi" + 0.007*"proyect" + 0.007*"titul" + 0.006*"copi" + 0.006*"solicit" + 0.006*"padron" + 0.006*"contrat" + 0.005*"beneficiari" + 0.005*"mexic"
Topic: 3 Word: 0.007*"program" + 0.007*"public" + 0.007*"presupuest" + 0.006*"solicit" + 0.006*"secretari" + 0.005*"informacion" + 0.005*"nacional" + 0.005*"educacion" + 0.005*"recurs" + 0.005*"servici"
Topic: 4 Word: 0.019*"medicament" + 0.018*"informacion" + 0.018*"sanitari" + 0.015*"articul" + 0.012*"registr" + 0.011*"present" + 0.010*"solicit" + 0.010*"activ" + 0.010*"adquir" + 0.008*"favor"
Topic: 5 Word: 0.010*"servidor" + 0.009

## Evaluación de performance LDA Bag of Words -- Clasificaremos un ejemplo 

In [51]:
processed_docs[4310]

['institucion',
 'solicit',
 'explic',
 'proced',
 'instal',
 'cafeteri',
 'plantel',
 'maquin',
 'caf',
 'refresc',
 'product',
 'industrializ',
 'solicit',
 'copi',
 'contrat',
 'conveni',
 'formaliz',
 'instalacion',
 'cit',
 'centr',
 'gananci',
 'institucion',
 'plantel',
 'dej',
 'consent',
 'tal',
 'form',
 'mercantil',
 'negoci',
 'autor',
 'consent',
 'destin',
 'recurs']

In [52]:
for index, score in sorted(lda_model[bow_corpus[4310]], 
                           key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score,
                                            lda_model.print_topic(index, 10)))



Score: 0.3845186233520508	 
Topic: 0.029*"contrat" + 0.023*"solicit" + 0.017*"proyect" + 0.014*"copi" + 0.012*"municipi" + 0.012*"servici" + 0.011*"empres" + 0.010*"fech" + 0.008*"mexic" + 0.007*"mont"

Score: 0.20881691575050354	 
Topic: 0.027*"solicit" + 0.017*"mont" + 0.017*"fech" + 0.015*"pag" + 0.014*"servici" + 0.013*"informacion" + 0.013*"concept" + 0.013*"trabaj" + 0.011*"bas" + 0.011*"tip"

Score: 0.1759869009256363	 
Topic: 0.027*"solicit" + 0.014*"document" + 0.012*"nacional" + 0.012*"copi" + 0.010*"institut" + 0.009*"director" + 0.009*"ofici" + 0.008*"direccion" + 0.008*"fech" + 0.008*"comision"

Score: 0.13493725657463074	 
Topic: 0.030*"public" + 0.019*"solicit" + 0.015*"federal" + 0.013*"secretari" + 0.013*"administr" + 0.012*"informacion" + 0.011*"servidor" + 0.011*"servici" + 0.008*"program" + 0.008*"materi"

Score: 0.08100435882806778	 
Topic: 0.036*"informacion" + 0.028*"solicit" + 0.014*"solucion" + 0.011*"acces" + 0.011*"document" + 0.010*"articul" + 0.009*"cas" +

In [53]:
for index, score in sorted(lda_model[bow_corpus[410]], 
                           key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score,
                                            lda_model.print_topic(index, 10)))




Score: 0.5499981641769409	 
Topic: 0.241*"solicitud" + 0.022*"salud" + 0.016*"informacion" + 0.014*"edad" + 0.012*"program" + 0.012*"person" + 0.011*"gener" + 0.011*"respuest" + 0.009*"solicit" + 0.009*"nin"

Score: 0.05000079795718193	 
Topic: 0.036*"informacion" + 0.028*"solicit" + 0.014*"solucion" + 0.011*"acces" + 0.011*"document" + 0.010*"articul" + 0.009*"cas" + 0.009*"transparent" + 0.009*"dich" + 0.008*"solicitud"

Score: 0.05000077560544014	 
Topic: 0.046*"informacion" + 0.038*"articul" + 0.029*"contrat" + 0.022*"licitacion" + 0.020*"transparent" + 0.019*"adquir" + 0.019*"acces" + 0.018*"proveedor" + 0.018*"medicament" + 0.018*"adquisicion"

Score: 0.05000004544854164	 
Topic: 0.029*"contrat" + 0.023*"solicit" + 0.017*"proyect" + 0.014*"copi" + 0.012*"municipi" + 0.012*"servici" + 0.011*"empres" + 0.010*"fech" + 0.008*"mexic" + 0.007*"mont"

Score: 0.05000004172325134	 
Topic: 0.049*"solicit" + 0.030*"registr" + 0.022*"informacion" + 0.020*"sanitari" + 0.018*"fech" + 0.016*"m

## Evaluación de performance LDA TF-IDF -- Clasificaremos un ejemplo 

In [54]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]],
                          key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}.".format(score,
                                             lda_model_tfidf.print_topic(index, 10)))


Score: 0.6823902726173401	 
Topic: 0.007*"program" + 0.007*"public" + 0.007*"presupuest" + 0.006*"solicit" + 0.006*"secretari" + 0.005*"informacion" + 0.005*"nacional" + 0.005*"educacion" + 0.005*"recurs" + 0.005*"servici".

Score: 0.19794321060180664	 
Topic: 0.019*"medicament" + 0.018*"informacion" + 0.018*"sanitari" + 0.015*"articul" + 0.012*"registr" + 0.011*"present" + 0.010*"solicit" + 0.010*"activ" + 0.010*"adquir" + 0.008*"favor".

Score: 0.09905120730400085	 
Topic: 0.010*"part" + 0.009*"mont" + 0.008*"gast" + 0.007*"diput" + 0.007*"pag" + 0.006*"electoral" + 0.006*"contrat" + 0.006*"fech" + 0.006*"solicit" + 0.006*"copi".


In [55]:
for index, score in sorted(lda_model_tfidf[bow_corpus[410]],
                          key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}.".format(score,
                                             lda_model_tfidf.print_topic(index, 10)))


Score: 0.549999475479126	 
Topic: 0.961*"solicitud" + 0.006*"informacion" + 0.001*"suplenci" + 0.001*"escane" + 0.001*"googl" + 0.001*"impos" + 0.001*"consupag" + 0.001*"crohn" + 0.000*"land" + 0.000*"postur".

Score: 0.050000160932540894	 
Topic: 0.019*"medicament" + 0.018*"informacion" + 0.018*"sanitari" + 0.015*"articul" + 0.012*"registr" + 0.011*"present" + 0.010*"solicit" + 0.010*"activ" + 0.010*"adquir" + 0.008*"favor".

Score: 0.05000007897615433	 
Topic: 0.012*"copi" + 0.011*"ofici" + 0.010*"vazquez" + 0.009*"nav" + 0.009*"contrat" + 0.006*"solicit" + 0.006*"servici" + 0.006*"consultor" + 0.005*"fech" + 0.005*"plag".

Score: 0.050000064074993134	 
Topic: 0.023*"adjunt" + 0.011*"medic" + 0.011*"ampar" + 0.011*"tribunal" + 0.010*"expedient" + 0.009*"juici" + 0.009*"administr" + 0.009*"circuit" + 0.008*"informacion" + 0.008*"document".

Score: 0.050000060349702835	 
Topic: 0.008*"informacion" + 0.008*"person" + 0.006*"delit" + 0.006*"edad" + 0.006*"solicit" + 0.006*"denunci" + 0.

# Guardamos como pickle

In [56]:
import pickle

In [57]:
with open('processed_docs_radicalizacion.pkl', 'wb') as f:
    pickle.dump(processed_docs, f)

In [58]:
with open('lda_model_bw_radicalizacion.pkl', 'wb') as f:
    pickle.dump(lda_model, f)

In [59]:
with open('lda_model_tfidf_radicalizacion.pkl', 'wb') as f:
    pickle.dump(lda_model_tfidf, f)

# Creamos una función para tokenizar, normalizar y lematizar.

In [9]:
import spacy
import nltk
from nltk import SnowballStemmer

In [10]:
nlp = spacy.load('es_core_news_sm')

In [11]:
def LematizarTexto(texto):
    
    doc = nlp(texto)
    words = [t.lemma_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and t.isalpha()]
    
    return lexical_tokens

In [12]:
texto

'a esta institucion solicito sea explicado el procedimiento el cual se instalaron cafeterias en los planteles al igual que maquinas de cafe refrescos y otros productos industrializados. tambien solicito copia de los contratos convenios a de los se formaliza instalacion en los citados centros . ganancias que a la institucion (mes por en uno de los planteles) deja el haber consentido la instalacion-operacion-funcionamiento de tales formas mercantiles. con fueron negociadas y que autoridades dieron consentimiento? y cual el destino de esos recursos?'

In [13]:
LematizarTexto(texto)

['institucion',
 'solicitar',
 'explicar',
 'procedimiento',
 'instalar',
 'cafeterias',
 'plantel',
 'maquinar',
 'cafe',
 'refresco',
 'producto',
 'industrializar',
 'solicitar',
 'copiar',
 'contrato',
 'convenio',
 'formalizar',
 'instalacion',
 'citar',
 'centro',
 'ganancia',
 'institucion',
 'plantel',
 'dejar',
 'consentir',
 'talar',
 'formar',
 'mercantil',
 'negociar',
 'autoridad',
 'consentimiento',
 'destinar',
 'recurso']

In [14]:
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool

In [15]:
cores = multiprocessing.cpu_count()
pool = ThreadPool(cores)
cores

12

In [16]:
processed_docs = pool.map(LematizarTexto, docs['descripcionsolicitud'])
pool.close()

In [17]:
processed_docs[:10]

[['copiar',
  'documento',
  'comportamiento',
  'historico',
  'tipicos',
  'comprender',
  'proyectar',
  'estudio',
  'cientificos',
  'variable',
  'presentarse',
  'pemex',
  'exploracion',
  'produccion',
  'elaborar',
  'evaluacion',
  'integral',
  'proyectar',
  'shcp',
  'incluir',
  'formar',
  'especificar',
  'produccion',
  'aceitar',
  'barril',
  'petrolear',
  'pozo',
  'considerar',
  'evaluacion',
  'integral',
  'proyectar',
  'declinacion',
  'anual',
  'petrolear',
  'pozo',
  'usar',
  'produccion',
  'preciar',
  'petrolear',
  'considerar',
  'responsable',
  'evaluacion',
  'integral',
  'proyectar',
  'costo',
  'operacion',
  'anual',
  'petrolear',
  'pozo',
  'considerar',
  'evaluacion',
  'integral',
  'proyectar'],
 ['informacion',
  'usar',
  'flujo',
  'efectivo',
  'pozo',
  'tipico',
  'aguar',
  'fria',
  'elaborar',
  'responder',
  'solicitud',
  'solicitar',
  'produccion',
  'aceitar',
  'considerar',
  'tabla',
  'costo',
  'capital',
  'usar'

In [18]:
import gensim

In [19]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [20]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 aceitar
1 anual
2 barril
3 cientificos
4 comportamiento
5 comprender
6 considerar
7 copiar
8 costo
9 declinacion
10 documento


In [21]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [22]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [23]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10,
                                      id2word=dictionary, passes=2,
                                      workers=cores)

In [24]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.024*"solicitar" + 0.022*"municipio" + 0.019*"santo" + 0.012*"copiar" + 0.011*"ubicar" + 0.010*"zona" + 0.008*"concesion" + 0.008*"mexico" + 0.008*"proyectar" + 0.008*"fecho"
Topic: 1 
Words: 0.016*"solicitar" + 0.015*"mexico" + 0.011*"informacion" + 0.011*"municipio" + 0.010*"nacional" + 0.010*"programar" + 0.007*"escuela" + 0.007*"ciudad" + 0.006*"aguar" + 0.006*"energia"
Topic: 2 
Words: 0.034*"solicitar" + 0.025*"informacion" + 0.017*"dependencia" + 0.016*"montar" + 0.012*"partir" + 0.012*"concepto" + 0.011*"basar" + 0.010*"fecho" + 0.010*"servicio" + 0.010*"presupuestar"
Topic: 3 
Words: 0.023*"solicitar" + 0.011*"informacion" + 0.010*"nacional" + 0.010*"federal" + 0.009*"secretario" + 0.009*"servidor" + 0.008*"direccion" + 0.008*"publicar" + 0.008*"fecho" + 0.007*"informar"
Topic: 4 
Words: 0.030*"solicitar" + 0.023*"solucion" + 0.015*"informacion" + 0.013*"proveedor" + 0.012*"adquisicion" + 0.011*"pemex" + 0.011*"licitacion" + 0.011*"nacional" + 0.011*"programa

In [25]:
for index, score in sorted(lda_model[bow_corpus[4310]], 
                           key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score,
                                            lda_model.print_topic(index, 10)))


Score: 0.354082316160202	 
Topic: 0.016*"solicitar" + 0.015*"mexico" + 0.011*"informacion" + 0.011*"municipio" + 0.010*"nacional" + 0.010*"programar" + 0.007*"escuela" + 0.007*"ciudad" + 0.006*"aguar" + 0.006*"energia"

Score: 0.23463356494903564	 
Topic: 0.036*"servicio" + 0.035*"contratar" + 0.026*"solicitar" + 0.025*"contrato" + 0.020*"copiar" + 0.014*"convenio" + 0.013*"empresa" + 0.013*"fecho" + 0.013*"informacion" + 0.010*"firmar"

Score: 0.2004830241203308	 
Topic: 0.037*"solicitar" + 0.023*"informacion" + 0.017*"fecho" + 0.015*"registrar" + 0.013*"sanitario" + 0.013*"tramitar" + 0.012*"federal" + 0.010*"presentar" + 0.008*"respuesta" + 0.008*"informar"

Score: 0.13379602134227753	 
Topic: 0.024*"personar" + 0.016*"solicitar" + 0.014*"social" + 0.012*"informacion" + 0.011*"fecho" + 0.009*"casar" + 0.008*"tipo" + 0.008*"seguro" + 0.008*"programar" + 0.008*"edad"

Score: 0.06220269575715065	 
Topic: 0.034*"solicitar" + 0.025*"informacion" + 0.017*"dependencia" + 0.016*"montar" + 

In [26]:
from gensim import corpora, models
from pprint import pprint

In [27]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [28]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                            num_topics=10,
                                            id2word=dictionary,
                                            passes=2,
                                            workers=cores)

In [29]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.022*"contratar" + 0.017*"contrato" + 0.017*"servicio" + 0.011*"empresa" + 0.010*"montar" + 0.008*"copiar" + 0.008*"licitacion" + 0.007*"dependencia" + 0.007*"solicitar" + 0.007*"contratacion"
Topic: 1 Word: 0.015*"medicar" + 0.008*"medicamento" + 0.008*"hospital" + 0.008*"unidad" + 0.007*"solicitar" + 0.007*"recetar" + 0.006*"paciente" + 0.006*"direccion" + 0.006*"correo" + 0.006*"informacion"
Topic: 2 Word: 0.971*"solicitud" + 0.001*"biotecnologicos" + 0.001*"apizaco" + 0.001*"celulas" + 0.001*"valsartan" + 0.000*"crohn" + 0.000*"paternidad" + 0.000*"pharmaceuticals" + 0.000*"artritis" + 0.000*"japon"
Topic: 3 Word: 0.011*"concesion" + 0.008*"copiar" + 0.007*"municipio" + 0.007*"santo" + 0.007*"solicitar" + 0.005*"ambiental" + 0.005*"proyectar" + 0.005*"aguar" + 0.005*"titular" + 0.005*"mexico"
Topic: 4 Word: 0.018*"sanitario" + 0.013*"medicamento" + 0.013*"informacion" + 0.011*"registrar" + 0.011*"cofepris" + 0.011*"tramitar" + 0.010*"presentar" + 0.009*"solicitar" +

In [30]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]],
                          key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}.".format(score,
                                             lda_model_tfidf.print_topic(index, 10)))


Score: 0.422720342874527	 
Topic: 0.011*"concesion" + 0.008*"copiar" + 0.007*"municipio" + 0.007*"santo" + 0.007*"solicitar" + 0.005*"ambiental" + 0.005*"proyectar" + 0.005*"aguar" + 0.005*"titular" + 0.005*"mexico".

Score: 0.33429965376853943	 
Topic: 0.022*"contratar" + 0.017*"contrato" + 0.017*"servicio" + 0.011*"empresa" + 0.010*"montar" + 0.008*"copiar" + 0.008*"licitacion" + 0.007*"dependencia" + 0.007*"solicitar" + 0.007*"contratacion".

Score: 0.1834556758403778	 
Topic: 0.006*"solicitar" + 0.005*"publicar" + 0.005*"informacion" + 0.005*"federal" + 0.005*"secretario" + 0.004*"programar" + 0.004*"nacional" + 0.004*"administrativo" + 0.004*"procedimiento" + 0.004*"fecho".

Score: 0.041839297860860825	 
Topic: 0.021*"informacion" + 0.008*"solicitar" + 0.007*"pemex" + 0.007*"energia" + 0.007*"municipio" + 0.005*"gasolina" + 0.005*"hidrocarburo" + 0.005*"mexico" + 0.005*"electrica" + 0.005*"produccion".


In [31]:
import pickle

In [32]:
with open('processed_docs_lematizacion.pkl', 'wb') as f:
    pickle.dump(processed_docs, f)

In [33]:
with open('lda_model_bw_lematizacion.pkl', 'wb') as f:
    pickle.dump(lda_model, f)

In [34]:
with open('lda_model_tfidf_lematizacion.pkl', 'wb') as f:
    pickle.dump(lda_model_tfidf, f)

## Lematizando y quitando palabras [solicitar, informacion, mexico, titular, copiar, dependencia, municipio, ...]  (Segunda iteración)

In [5]:
import spacy
import nltk
from nltk import SnowballStemmer

In [6]:
nlp = spacy.load('es_core_news_sm')

In [7]:
def LematizarTexto(texto):
    
    doc = nlp(texto)
    words = [t.lemma_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and t.isalpha()]
    
    return lexical_tokens

In [10]:
banned = ['solicitar', 'solicito', 'solicitamos', 'solicite', 'informacion', 'mexico', 'titular', 
          'titulo', 'copiar', 'copie', 'copiamos', 'dependencia', 'municipio', 'tramitar', 'contratar', 
          'contrato', 'servicio', 'tramitar', 'tramite', 'proyectar', 'proyectamos', 'proyecte', 'proyeccion', 
          'nacional', 'administrativo', 'presupuestar', 'presupuesto', 'correo', 'pemex', 'programar', 'programa',
         'programacion', 'empresa']

In [11]:
f = lambda x: ' '.join([item for item in x.split() if item not in banned])
df['descripcionsolicitud'] = df['descripcionsolicitud'].apply(f)

In [12]:
data = df[['descripcionsolicitud']]
data['index'] = data.index
docs = data

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
texto = docs[docs['index'] == 4310].values[0][0]
texto

'a esta institucion sea explicado el procedimiento el cual se instalaron cafeterias en los planteles al igual que maquinas de cafe refrescos y otros productos industrializados. tambien copia de los contratos convenios a de los se formaliza instalacion en los citados centros . ganancias que a la institucion (mes por en uno de los planteles) deja el haber consentido la instalacion-operacion-funcionamiento de tales formas mercantiles. con fueron negociadas y que autoridades dieron consentimiento? y cual el destino de esos recursos?'

In [14]:
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool

In [15]:
cores = multiprocessing.cpu_count()
pool = ThreadPool(cores)
cores

12

In [16]:
processed_docs = pool.map(LematizarTexto, docs['descripcionsolicitud'])
pool.close()

In [17]:
import gensim

In [18]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [19]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [20]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [21]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10,
                                      id2word=dictionary, passes=2,
                                      workers=cores)

In [22]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.450*"solicitud" + 0.030*"respuesta" + 0.024*"adjuntar" + 0.012*"informacion" + 0.011*"personal" + 0.010*"reglamentar" + 0.007*"salud" + 0.007*"activar" + 0.007*"proteccion" + 0.007*"esperar"
Topic: 1 
Words: 0.056*"medicar" + 0.031*"medicamento" + 0.026*"hospital" + 0.021*"clavar" + 0.021*"unidad" + 0.020*"atencion" + 0.017*"paciente" + 0.017*"recetar" + 0.016*"fecho" + 0.015*"nivel"
Topic: 2 
Words: 0.011*"copiar" + 0.008*"amparar" + 0.008*"expedientar" + 0.008*"tribunal" + 0.007*"federal" + 0.007*"fecho" + 0.007*"presentar" + 0.007*"materia" + 0.007*"juicio" + 0.006*"instituto"
Topic: 3 
Words: 0.034*"medicamento" + 0.027*"adquirir" + 0.024*"articular" + 0.023*"licitacion" + 0.023*"proveedor" + 0.022*"adquisicion" + 0.020*"favor" + 0.020*"clavar" + 0.017*"presentar" + 0.015*"producto"
Topic: 4 
Words: 0.017*"solucion" + 0.016*"partir" + 0.012*"personar" + 0.011*"entidad" + 0.010*"programar" + 0.009*"poblacion" + 0.008*"social" + 0.008*"destinar" + 0.007*"solicitar"

In [23]:
for index, score in sorted(lda_model[bow_corpus[4310]], 
                           key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score,
                                            lda_model.print_topic(index, 10)))


Score: 0.4691656231880188	 
Topic: 0.015*"montar" + 0.009*"recurso" + 0.009*"educacion" + 0.008*"apoyar" + 0.007*"pagar" + 0.007*"instituto" + 0.007*"concepto" + 0.007*"fecho" + 0.007*"secretario" + 0.007*"escuela"

Score: 0.20974045991897583	 
Topic: 0.029*"fecho" + 0.025*"contrato" + 0.024*"solicitar" + 0.024*"sanitario" + 0.021*"registrar" + 0.016*"servicio" + 0.013*"copiar" + 0.011*"presentar" + 0.010*"comision" + 0.010*"informar"

Score: 0.1464967131614685	 
Topic: 0.017*"solucion" + 0.016*"partir" + 0.012*"personar" + 0.011*"entidad" + 0.010*"programar" + 0.009*"poblacion" + 0.008*"social" + 0.008*"destinar" + 0.007*"solicitar" + 0.007*"favor"

Score: 0.1225404292345047	 
Topic: 0.450*"solicitud" + 0.030*"respuesta" + 0.024*"adjuntar" + 0.012*"informacion" + 0.011*"personal" + 0.010*"reglamentar" + 0.007*"salud" + 0.007*"activar" + 0.007*"proteccion" + 0.007*"esperar"

Score: 0.036339834332466125	 
Topic: 0.012*"copiar" + 0.009*"santo" + 0.009*"fecho" + 0.008*"proyectar" + 0.007

In [24]:
from gensim import corpora, models
from pprint import pprint

In [25]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [26]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                            num_topics=10,
                                            id2word=dictionary,
                                            passes=2,
                                            workers=cores)

In [27]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.963*"solicitud" + 0.002*"militante" + 0.001*"rastro" + 0.001*"incumplio" + 0.001*"prevalencia" + 0.000*"dengue" + 0.000*"colosio" + 0.000*"hemodinamia" + 0.000*"donaldo" + 0.000*"cessna"
Topic: 1 Word: 0.008*"copiar" + 0.005*"fecho" + 0.005*"facturar" + 0.005*"contrato" + 0.004*"informar" + 0.003*"montar" + 0.003*"lopez" + 0.003*"servicio" + 0.003*"pagar" + 0.003*"cargar"
Topic: 2 Word: 0.015*"licitacion" + 0.013*"contrato" + 0.012*"medicamento" + 0.009*"servicio" + 0.009*"adjudicacion" + 0.009*"directo" + 0.008*"adquisicion" + 0.008*"solicitar" + 0.008*"proveedor" + 0.007*"favor"
Topic: 3 Word: 0.012*"sanitario" + 0.009*"registrar" + 0.009*"solicitar" + 0.007*"cofepris" + 0.007*"fecho" + 0.006*"presentar" + 0.006*"transparencia" + 0.006*"acceso" + 0.005*"comision" + 0.005*"activar"
Topic: 4 Word: 0.007*"personar" + 0.006*"programar" + 0.006*"desglosar" + 0.006*"mujer" + 0.005*"entidad" + 0.005*"edad" + 0.005*"delito" + 0.004*"federativo" + 0.004*"poblacion" + 0.004*"t

In [28]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]],
                          key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}.".format(score,
                                             lda_model_tfidf.print_topic(index, 10)))


Score: 0.6410174369812012	 
Topic: 0.007*"personar" + 0.006*"programar" + 0.006*"desglosar" + 0.006*"mujer" + 0.005*"entidad" + 0.005*"edad" + 0.005*"delito" + 0.004*"federativo" + 0.004*"poblacion" + 0.004*"tipo".

Score: 0.17656007409095764	 
Topic: 0.015*"licitacion" + 0.013*"contrato" + 0.012*"medicamento" + 0.009*"servicio" + 0.009*"adjudicacion" + 0.009*"directo" + 0.008*"adquisicion" + 0.008*"solicitar" + 0.008*"proveedor" + 0.007*"favor".

Score: 0.09025648981332779	 
Topic: 0.010*"trabajador" + 0.007*"educacion" + 0.007*"sindicato" + 0.005*"diputar" + 0.005*"secretario" + 0.005*"electoral" + 0.005*"instituto" + 0.005*"salariar" + 0.005*"copiar" + 0.004*"pagar".

Score: 0.07338825613260269	 
Topic: 0.007*"concesion" + 0.007*"amparar" + 0.007*"tribunal" + 0.006*"copiar" + 0.006*"circuito" + 0.006*"proyectar" + 0.005*"materia" + 0.005*"juicio" + 0.005*"colegiar" + 0.005*"expedientar".


In [29]:
import pickle

In [30]:
with open('processed_docs_lematizacion.pkl', 'wb') as f:
    pickle.dump(processed_docs, f)

In [31]:
with open('lda_model_bw_lematizacion.pkl', 'wb') as f:
    pickle.dump(lda_model, f)

In [32]:
with open('lda_model_tfidf_lematizacion.pkl', 'wb') as f:
    pickle.dump(lda_model_tfidf, f)